In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("wc_event_data.csv")
df.head()

C:\Users\Omar\AppData\Local\Temp\ipykernel_31712\1576540362.py:1: DtypeWarning: Columns (55,68,72,79,84,92,93,94,96,97,98,99,100,101,102,103,105,106,112,113,115,116,118,119,120,121,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("wc_event_data.csv")


,id,index,period,timestamp,minute,second,possession,duration,type_id,type_name,...,shot_saved_to_post,goalkeeper_shot_saved_to_post,half_start_late_video_start,shot_saved_off_target,goalkeeper_shot_saved_off_target,goalkeeper_success_in_play,shot_redirect,goalkeeper_lost_in_play,x,y
0,093f898d-33b1-4425-b591-37dd9c9bf70b,1,1,00:00:00.000,0,0,1,0.000000,35,Starting XI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,79a9efbb-b5e5-45b0-983d-8729bcc4a0e1,2,1,00:00:00.000,0,0,1,0.000000,35,Starting XI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c4cdfc71-b9f4-4751-a714-e816c1419457,3,1,00:00:00.000,0,0,1,0.000000,18,Half Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51f295b8-da96-428e-ab47-90f380f8cf53,4,1,00:00:00.000,0,0,1,0.000000,18,Half Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4acb4fd2-f46a-4d73-993c-e06597873924,5,1,00:00:00.521,0,0,2,1.220932,30,Pass,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,40.1


In [3]:
crosses = df[(df['pass_cross'] == True) & pd.isna(df['pass_outcome_name'])]
crosses = crosses.groupby('player_name').size().reset_index(name='successful_crosses')
cross_sorted = crosses.sort_values(by='successful_crosses', ascending=False)
print(cross_sorted)

                           player_name  successful_crosses
127               Kylian Mbappé Lottin                  13
169                    Ousmane Dembélé                  10
228    Ángel Fabián Di María Hernández                   9
94                        Ivan Perišić                   7
212    Theo Bernard François Hernández                   6
..                                 ...                 ...
98     Jeremy Leonel Sarmiento Morante                   1
99                 Jerome Ngom Mbekeli                   1
103                        Jordan Ayew                   1
109                João Félix Sequeira                   1
230  Éverton Augusto de Barros Ribeiro                   1

[231 rows x 2 columns]


In [4]:
# Filter the DataFrame for rows where the position_name is 'Striker'
striker = df[df['position_name'] == 'Center Forward']

# Filter the striker DataFrame for total defensive actions
defensive_striker_actions = striker[striker['type_name'].isin(['Clearance', 'Block', 'Duel', 'Interception'])]
# Group by player_name and count the total actions for each player
player_defensive_actions = defensive_striker_actions.groupby('player_name').size().reset_index(name='successful_actions')

player_defensive_actions_sorted = player_defensive_actions.sort_values(by='successful_actions', ascending=False)

print(player_defensive_actions_sorted)


                player_name  successful_actions
78        Youssef En-Nesyri                  41
62           Olivier Giroud                  30
27               Harry Kane                  27
30             Issam Jebali                  26
66   Richarlison de Andrade                  24
..                      ...                 ...
46  Lautaro Javier Martínez                   1
1      Abdulrahman Al-Obood                   1
38           Jonathan David                   1
35         Jewison Bennette                   1
56            Memphis Depay                   1

[80 rows x 2 columns]


In [5]:
cb = df[df['position_name'] == 'Center Back']

# Filter the cb DataFrame for successful dribbles
cb_dribbles = cb[(cb['type_name'] == 'Dribble') & (cb['dribble_outcome_name'] == 'Complete')]

# Group by player_name and count the successful dribbles for each player
cb_dribbles = cb_dribbles.groupby('player_name').size().reset_index(name='successful_dribbles')

print(cb_dribbles)


       player_name  successful_dribbles
0  Alexander Djiku                    1
1     Maya Yoshida                    1
2  Virgil van Dijk                    1


## Top teams for average of total xG/match

In [6]:
shots = df[df['type_name'] == 'Shot']
xg_per_match = shots.groupby(['team_name','match_id']).agg(total_xG=('shot_statsbomb_xg','sum')).reset_index()
avg_xg = xg_per_match.groupby(['team_name']).agg(Avg_xG=('total_xG','mean')).reset_index()
avg_xg.sort_values(by='Avg_xG',ascending=False).head(10)

,team_name,Avg_xG
0,Argentina,2.998349
12,Germany,2.743693
3,Brazil,2.727319
11,France,2.136551
7,Croatia,1.881386
15,Japan,1.845893
18,Netherlands,1.782257
26,Spain,1.776157
10,England,1.747171
27,Switzerland,1.522629


## Top players to pass with the weak foot

In [7]:
passes = df[df['type_name'] == 'Pass']
players_foot = passes[passes['pass_body_part_name'].isin(['Left Foot','Right Foot'])].groupby(['player_name','pass_body_part_name']).agg(body_part_count=('id','count')).reset_index()

In [8]:
players_foot

,player_name,pass_body_part_name,body_part_count
0,Aaron Mooy,Left Foot,48
1,Aaron Mooy,Right Foot,161
2,Aaron Ramsey,Left Foot,27
3,Aaron Ramsey,Right Foot,77
4,Abdelhamid Sabiri,Left Foot,1
...,...,...,...
1282,Éverton Augusto de Barros Ribeiro,Right Foot,6
1283,Óscar Esau Duarte Gaitán,Left Foot,23
1284,Óscar Esau Duarte Gaitán,Right Foot,65
1285,İlkay Gündoğan,Left Foot,50


In [9]:
players_foot['rank'] = players_foot.groupby(['player_name'])['body_part_count'].rank(method='max', ascending=False)
players_foot

,player_name,pass_body_part_name,body_part_count,rank
0,Aaron Mooy,Left Foot,48,2.0
1,Aaron Mooy,Right Foot,161,1.0
2,Aaron Ramsey,Left Foot,27,2.0
3,Aaron Ramsey,Right Foot,77,1.0
4,Abdelhamid Sabiri,Left Foot,1,2.0
...,...,...,...,...
1282,Éverton Augusto de Barros Ribeiro,Right Foot,6,2.0
1283,Óscar Esau Duarte Gaitán,Left Foot,23,2.0
1284,Óscar Esau Duarte Gaitán,Right Foot,65,1.0
1285,İlkay Gündoğan,Left Foot,50,2.0


In [10]:
strong_foot = players_foot[players_foot['rank'] == 1]
weak_foot = players_foot[players_foot['rank'] == 2]

In [11]:
merged = pd.merge(left=strong_foot[['player_name','pass_body_part_name','body_part_count']]
                  ,right=weak_foot[['player_name','body_part_count']]
                  ,how='left'
                  ,on='player_name'
                  ,suffixes=('_strong','_weak')).rename(columns={'pass_body_part_name':'strong_foot'})
merged['weak_foot%'] = merged['body_part_count_weak'] / (merged['body_part_count_strong']+merged['body_part_count_weak'])
merged = merged.sort_values(by='weak_foot%',ascending=False)

In [12]:
merged.head(10)

,player_name,strong_foot,body_part_count_strong,body_part_count_weak,weak_foot%
494,Pape Abou Cissé,Right Foot,19,18.0,0.486486
654,Yuto Nagatomo,Right Foot,45,41.0,0.476744
491,Ousmane Dembélé,Left Foot,95,86.0,0.475138
580,Stefan Mitrović,Right Foot,9,8.0,0.470588
43,Almoez Ali Zainalabiddin Abdulla,Right Foot,23,20.0,0.465116
537,Romario Andrés Ibarra Mina,Left Foot,7,6.0,0.461538
270,Ivan Perišić,Right Foot,125,106.0,0.458874
651,Youstin Delfin Salas Gómez,Left Foot,6,5.0,0.454545
516,Raphaël Varane,Right Foot,146,119.0,0.449057
47,Anass Zaroury,Right Foot,7,5.0,0.416667
